In [30]:
// Inverse Discrete Fourier Transforms in Rust

// consider the frequencies (1+0i, 1+0i, 0, 0)
// we should get a circle one unit to the right of the origin
// 2+0i, 1+1i, 0+0i, 1-1i
// :dep rustfft = "6.0"
extern crate rustfft;
use rustfft::{FftPlanner, num_complex::Complex};
use std::sync::Arc;

fn ifft(freqs: Vec<Complex<f32>>) -> Vec<Complex<f32>> {
    // copy the frequences
    let mut freqs = freqs.clone();
    // create planner and plan an inverse fft
    let mut planner = FftPlanner::new();
    let ifft = planner.plan_fft_inverse(freqs.len());
    // perform inverse fft in place
    ifft.process(&mut freqs);
    // don't need to normalize
    freqs 
}

let example_freqs = vec![
    Complex::new(1.0, 0.0),
    Complex::new(1.0, 0.0),
    Complex::new(0.0, 0.0),
    Complex::new(0.0, 0.0),
];

use std::f32::consts::PI;

fn single_freq_tseries(k: i32, c_k: f32, N: i32) -> Vec<Complex<f32>> {
    let signal: Vec<Complex<f32>> = (0..N)
        .map(|n| {
            let t = n as f32 / N as f32; // time corresponding to sample n
            Complex::new(0.0, 2.0 * PI * (k as f32) * t).exp() * c_k
        })
        .collect();

    signal
}

fn create_lines(freqs: Vec<f32>, k: i32, line_length: f32) -> Vec<f32> {
    let K = freqs.len() as i32;

    // Map frequencies to complex numbers with an imaginary part of 0
    let freqs: Vec<Complex<f32>> = freqs.into_iter().map(|c_k| Complex::new(c_k, 0.0)).collect();

    // Centers are obtained from the inverse FFT
    let centers: Vec<Complex<f32>> = ifft(freqs);

    // Get the delta values based on the frequency series
    let delta: Vec<Complex<f32>> = single_freq_tseries(k, line_length / 2.0, K);

    // Create the lines by calculating the start and end points for each line
    let mut lines = Vec::with_capacity(4 * K as usize);

    for i in 0..K as usize {
        lines.push(centers[i].re - delta[i].re);
        lines.push(centers[i].im - delta[i].im);
        lines.push(centers[i].re + delta[i].re);
        lines.push(centers[i].im + delta[i].im);
    }

    lines
}
// single_freq_tseries(1, 1.0, 4)
// ifft(example_freqs)
let example_freqs = vec![1.0, 1.0, 0.0, 0.0];
let lines = create_lines(example_freqs, 1, 0.1);
println!("{:?}", lines)

[1.95, 0.0, 2.05, 0.0, 1.0, 0.95, 1.0, 1.05, 0.05, 4.371139e-9, -0.05, -4.371139e-9, 1.0, -0.95, 1.0, -1.05]


()